In [1]:
# Library imports
import tensorflow as tf
import numpy as np
keras = tf.keras
import matplotlib.pyplot as plt
import os
import sys
import contextlib

In [2]:
# Add src directory to path for local imports
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Local imports
from utils import gpu_grow_memory

In [3]:
gpu_grow_memory()

In [4]:
# Some helper code to demonstrate the kinds of errors you might encounter.
@contextlib.contextmanager
def assert_raises(error_class):
  try:
    yield
  except error_class as e:
    print('Caught expected exception \n  {}: {}'.format(error_class, e))
  except Exception as e:
    print('Got unexpected exception \n  {}: {}'.format(type(e), e))
  else:
    raise Exception('Expected {} to be raised but no error was raised!'.format(
        error_class))

In [5]:
@tf.function
def add(a, b):
    return a + b

In [6]:
add(tf.ones([2,2]), tf.ones([2,2]))

<tf.Tensor: id=14, shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

In [8]:
v = tf.Variable(1.0)
with tf.GradientTape() as tape:
    result = add(v, 1.0)
tape.gradient(result, v)

<tf.Tensor: id=61, shape=(), dtype=float32, numpy=1.0>

In [9]:
@tf.function
def dense_layer(x, w, b):
    return tf.add(tf.matmul(x, w), b)

In [12]:
dense_layer(tf.ones([3,2]), tf.ones([2,2]), tf.ones([2]))

<tf.Tensor: id=87, shape=(3, 2), dtype=float32, numpy=
array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)>

In [13]:
@tf.function
def double(a):
    print(f'Tracing with ', a)
    return a + a

In [14]:
double(tf.constant(1))

Tracing with  Tensor("a:0", shape=(), dtype=int32)


<tf.Tensor: id=96, shape=(), dtype=int32, numpy=2>

In [15]:
double(tf.constant(1.0))

Tracing with  Tensor("a:0", shape=(), dtype=float32)


<tf.Tensor: id=104, shape=(), dtype=float32, numpy=2.0>

In [16]:
double(tf.constant('a'))

Tracing with  Tensor("a:0", shape=(), dtype=string)


<tf.Tensor: id=112, shape=(), dtype=string, numpy=b'aa'>

In [20]:
print('Get concrete trace')
double_strings = double.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.string))

print('Executing traced function')
print(double_strings(tf.constant('a')))
print(double_strings(a=tf.constant('b')))

print(f'Using incompatible types leads to an error')
with assert_raises(tf.errors.InvalidArgumentError):
    print(double_strings(tf.constant(1)))

Get concrete trace
Executing traced function
tf.Tensor(b'aa', shape=(), dtype=string)
tf.Tensor(b'bb', shape=(), dtype=string)
Using incompatible types leads to an error
Caught expected exception 
  <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>: cannot compute __inference_double_118 as input #0(zero-based) was expected to be a string tensor but is a int32 tensor [Op:__inference_double_118]


In [21]:
@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
def collatz(x):
    print(f'Tracing with ', x)
    return tf.where(tf.equal(x%2, 0), x//2, 3*x+1)

In [22]:
collatz(tf.constant([1,2,3,4,5,6,7,8,9,10]))

Tracing with  Tensor("x:0", shape=(None,), dtype=int32)


<tf.Tensor: id=161, shape=(10,), dtype=int32, numpy=array([ 4,  1, 10,  2, 16,  3, 22,  4, 28,  5])>

In [23]:
with assert_raises(ValueError):
    collatz(tf.constant([[1,2], [3,4]]))

Caught expected exception 
  <class 'ValueError'>: Python inputs incompatible with input_signature: inputs ((<tf.Tensor: id=163, shape=(2, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4]])>,)), input_signature ((TensorSpec(shape=(None,), dtype=tf.int32, name=None),))


In [24]:
def train_one_step():
    pass

@tf.function
def train(num_steps):
    print(f'Tracing with num_steps={num_steps}')
    for _ in tf.range(num_steps):
        train_one_step()

train(num_steps=10)
train(num_steps=20)

Tracing with num_steps=10
Tracing with num_steps=20


In [25]:
train(num_steps=tf.constant(10))
train(num_steps=tf.constant(20))

Tracing with num_steps=Tensor("num_steps:0", shape=(), dtype=int32)


In [26]:
@tf.function
def f(x):
    print('Traced with', x)
    tf.print('Executed with', x)

f(1)
f(1)
f(2)

Traced with 1
Executed with 1
Executed with 1
Traced with 2
Executed with 2


In [27]:
external_list = []

def side_effect(x):
    print(f'Python side effect')
    external_list.append(x)
    
@tf.function
def f(x):
    tf.py_function(side_effect, inp=[x], Tout=[])
    
f(1)
f(1)
f(1)
external_list

W0613 11:49:55.761207 18156 backprop.py:842] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.int32
W0613 11:49:55.766216  6820 backprop.py:842] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.int32
W0613 11:49:55.771209  6820 backprop.py:842] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.int32


Python side effect
Python side effect
Python side effect


[<tf.Tensor: id=330, shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: id=331, shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: id=332, shape=(), dtype=int32, numpy=1>]

In [34]:
def measure_graph_size(f, *args):
    g = f.get_concrete_function(*args).graph
    arg_str = ', '.join(map(str, args))
    print(f'{f.__name__}({arg_str})) contains {len(g.as_graph_def().node)} nodes.')

In [35]:
@tf.function
def train(dataset):
    loss = tf.constant(0)
    for x, y in dataset:
        loss += tf.abs(y-x)
    return loss

In [36]:
small_data = [(1,1)] * 2
big_data = [(1,1)] * 10

measure_graph_size(train, small_data)
measure_graph_size(train, big_data)

train([(1, 1), (1, 1)])) contains 8 nodes.
train([(1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)])) contains 32 nodes.


In [37]:
measure_graph_size(train, tf.data.Dataset.from_generator(lambda: small_data, (tf.int32, tf.int32)))
measure_graph_size(train, tf.data.Dataset.from_generator(lambda: big_data, (tf.int32, tf.int32)))

W0613 14:10:42.061657 16864 deprecation.py:323] From C:\Anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:504: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


train(<DatasetV1Adapter shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>)) contains 4 nodes.
train(<DatasetV1Adapter shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>)) contains 4 nodes.


In [38]:
v = tf.Variable(1.0)
@tf.function
def f(x):
    return v.assign_add(x)

print(f(1.0))
print(f(2.0))

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [39]:
class C: pass
obj = C()
obj.v = None

@tf.function
def g(x):
    if obj.v is None:
        obj.v = tf.Variable(1.0)
    return obj.v.assign_add(x)

print(g(1.0))
print(g(2.0))

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [40]:
state = []

@tf.function
def fn(x):
    if not state:
        state.append(tf.Variable(2.0 * x))
        state.append(tf.Variable(state[0] * 3.0))
    return state[0] * x * state[1]

print(fn(tf.constant(1.0)))
print(fn(tf.constant(3.0)))

tf.Tensor(12.0, shape=(), dtype=float32)
tf.Tensor(36.0, shape=(), dtype=float32)


In [43]:
def test_tf_cond(f, *args):
    g = f.get_concrete_function(*args).graph
    if any(node.name == 'cond' for node in g.as_graph_def().node):
        print(f'{f.__name__} uses tf.cond')
    else:
        print(f'{f.__name__} executes normally')
        
@tf.function
def hyperparam_cond(x, training=True):
    if training:
        x = tf.nn.dropout(x, rate=0.5)
    return x

@tf.function
def maybe_tensor_cond(x):
    if x < 0:
        x = -x
    return x

test_tf_cond(hyperparam_cond, tf.ones([1], dtype=tf.float32))
test_tf_cond(maybe_tensor_cond, tf.constant(-1))
test_tf_cond(maybe_tensor_cond, (-1))

hyperparam_cond executes normally
maybe_tensor_cond uses tf.cond
maybe_tensor_cond executes normally


In [44]:
def test_dynamically_unrolled(f, *args):
  g = f.get_concrete_function(*args).graph
  if any(node.name == 'while' for node in g.as_graph_def().node):
    print("{}({}) uses tf.while_loop.".format(
        f.__name__, ', '.join(map(str, args))))
  elif any(node.name == 'ReduceDataset' for node in g.as_graph_def().node):
    print("{}({}) uses tf.data.Dataset.reduce.".format(
        f.__name__, ', '.join(map(str, args))))
  else:
    print("{}({}) gets unrolled.".format(
        f.__name__, ', '.join(map(str, args))))

In [47]:
@tf.function
def for_in_range():
    x = 0
    for i in range(5):
        x += i
    return x

test_dynamically_unrolled(for_in_range)

for_in_range() gets unrolled.


In [48]:
@tf.function
def for_in_tfrange():
    x = tf.constant(0, dtype=tf.int32)
    for i in tf.range(5):
         x += i
    return x

test_dynamically_unrolled(for_in_tfrange)

for_in_tfrange() uses tf.while_loop.


In [51]:
@tf.function
def for_in_tfdataset():
    x = tf.constant(0, dtype=tf.int64)
    for i in tf.data.Dataset.range(5):
        x += i
    return x

test_dynamically_unrolled(for_in_tfdataset)

for_in_tfdataset() uses tf.data.Dataset.reduce.
